In [28]:
import numpy as np

In [29]:
train_x = np.loadtxt('pp_train_x.csv', delimiter=",")

In [30]:
train_y = np.loadtxt('train_y.csv', delimiter=",")

In [31]:
test_x = np.loadtxt('pp_test_x.csv', delimiter=",")

In [32]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils import data
import torchvision.utils as vutils
from network import Network
import config as cfg
import logging
import os

In [43]:
def calc_acc(x, y):
    x = torch.max(x, dim=-1)[1]
    accuracy = sum(x == y) / x.size(0)
    return accuracy

logging.getLogger().setLevel(logging.INFO)
logging.info('Mode: %s' % cfg.mode)
if not os.path.exists(cfg.model_dir):
    os.mkdir(cfg.model_dir)
if not os.path.exists(cfg.transform_img_dir):
    os.mkdir(cfg.transform_img_dir)

net = Network(mode=cfg.mode)
if torch.cuda.is_available():
    net.cuda(cfg.cuda_num)

opt = torch.optim.Adam(net.parameters(), lr=cfg.LR)
loss_func = nn.CrossEntropyLoss()

def to_torch_format(img, target, data_type=''):
    '''
    :param img: (b, l)
    :param target: (b, 1)
    :param data_type: string
    :return: (b, c, h, w), (b)
    '''
    batch_size = img.size(0)
    img = img.view(batch_size, cfg.channel, cfg.height, cfg.width)
    target = target.view(batch_size)
    print(data_type, img.size(), target.size())
    return img, target

t_X = train_x[:45000]
t_y = train_y[:45000]

v_X = train_x[-5000:]
v_y = train_y[-5000:]

X_train = torch.from_numpy(t_X).float()
y_train = torch.from_numpy(t_y).long()

X_test = torch.from_numpy(v_X).float()
y_test = torch.from_numpy(v_y).long()

X_train, y_train = to_torch_format(X_train, y_train, data_type='Training')
X_test, y_test = to_torch_format(X_test, y_test, data_type='Testing')

train_dataset = data.TensorDataset(data_tensor=X_train, target_tensor=y_train)
train_loader = data.DataLoader(dataset=train_dataset, batch_size=cfg.train_batch_size, shuffle=True)

test_dataset = data.TensorDataset(data_tensor=X_test, target_tensor=y_test)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=cfg.test_batch_size)

train_batch_nb = len(train_loader)
test_batch_nb = len(test_loader)


# train_dataset = data.TensorDataset(data_tensor=train_x, target_tensor=np.array(map(lambda y: str(y),train_y)))
# test_dataset = data.TensorDataset(data_tensor=X_test, target_tensor=y_test)
# train_loader = data.DataLoader(dataset=train_dataset, batch_size=cfg.train_batch_size, shuffle=True)
# test_loader = data.DataLoader(dataset=test_dataset, batch_size=cfg.test_batch_size)


INFO:root:Mode: stn


Training torch.Size([45000, 1, 64, 64]) torch.Size([45000])
Testing torch.Size([5000, 1, 64, 64]) torch.Size([5000])


In [ ]:
for epoch_idx in range(cfg.epoch):
    # ========================== Training Model =============================
    net.train()
#     for batch_idx in range(len(train_x)):
    for batch_idx, (train_img, train_target) in enumerate(train_loader):
#         train_img = Variable(train_x[batch_idx])
#         train_target = Variable(train_y[batch_idx])
        train_img = Variable(train_img)
        train_target = Variable(train_target)

        if torch.cuda.is_available():
            train_img = train_img.cuda(cfg.cuda_num)
            train_target = train_target.cuda(cfg.cuda_num)

        _, predict = net(train_img)

        loss = loss_func(predict, train_target)
        net.zero_grad()
        loss.backward()
        opt.step()

        acc = calc_acc(predict.cpu().data, train_target.cpu().data)
        
        # if batch_idx % cfg.show_train_result_every_batch == 0:
        #     logging.info('epoch[%d/%d] batch[%d/%d] loss:%.4f acc:%.4f'
        #                  % (epoch_idx, cfg.epoch, batch_idx, train_batch_nb, loss.data[0], acc))

#     # ========================== Testing Model =============================
    if (epoch_idx + 1) % cfg.test_every_epoch == 0:
        net.eval()
        total_loss = 0
        total_acc = 0
#         for batch_idx in range(len(test_x)):
        for batch_idx, (test_img, test_target) in enumerate(test_loader):
            batch_size = test_img.size(0)

            test_img = Variable(test_img, volatile=True)
            test_target = Variable(test_target, volatile=True)

            if torch.cuda.is_available():
                test_img = test_img.cuda(cfg.cuda_num)
                test_target = test_target.cuda(cfg.cuda_num)

            transform_img, predict = net(test_img)

            loss = loss_func(predict, test_target)
            acc = calc_acc(predict.cpu().data, test_target.cpu().data)

            total_loss += loss
            total_acc += acc

            if cfg.mode == 'stn':
                img_list = []
                for idx in range(batch_size):
                    img_list.append(test_img[idx])
                    img_list.append(transform_img[idx])
                output_img = torch.stack(img_list)

                vutils.save_image(output_img.data, os.path.join(cfg.transform_img_dir, '%d.png' % batch_idx),
                                  nrow=20)

        mean_loss = total_loss / test_batch_nb
        mean_acc = total_acc / test_batch_nb
        logging.info('========= Testing: epoch[%d/%d] loss:%.4f acc:%.4f' % (epoch_idx, cfg.epoch, mean_loss.data[0], mean_acc))

    if (epoch_idx + 1) % cfg.save_model_every_epoch == 0:
        state_dict = net.state_dict()
        torch.save(state_dict, os.path.join(cfg.model_dir, cfg.model_name % cfg.mode))

INFO:root:========= Testing: epoch[0/200] loss:0.4094 acc:0.8918
INFO:root:========= Testing: epoch[1/200] loss:0.3448 acc:0.9086
INFO:root:========= Testing: epoch[2/200] loss:0.3187 acc:0.9170
INFO:root:========= Testing: epoch[3/200] loss:0.2986 acc:0.9215
INFO:root:========= Testing: epoch[4/200] loss:0.2844 acc:0.9283
INFO:root:========= Testing: epoch[5/200] loss:0.2666 acc:0.9307
INFO:root:========= Testing: epoch[6/200] loss:0.2679 acc:0.9279
INFO:root:========= Testing: epoch[7/200] loss:0.2591 acc:0.9328
INFO:root:========= Testing: epoch[8/200] loss:0.2565 acc:0.9326
INFO:root:========= Testing: epoch[9/200] loss:0.2517 acc:0.9350
INFO:root:========= Testing: epoch[10/200] loss:0.2409 acc:0.9369
INFO:root:========= Testing: epoch[11/200] loss:0.2394 acc:0.9404
INFO:root:========= Testing: epoch[12/200] loss:0.2426 acc:0.9373
INFO:root:========= Testing: epoch[13/200] loss:0.2322 acc:0.9377
INFO:root:========= Testing: epoch[14/200] loss:0.2291 acc:0.9400
INFO:root:========= 

INFO:root:========= Testing: epoch[124/200] loss:0.2074 acc:0.9521
INFO:root:========= Testing: epoch[125/200] loss:0.2055 acc:0.9523
INFO:root:========= Testing: epoch[126/200] loss:0.2256 acc:0.9498
INFO:root:========= Testing: epoch[127/200] loss:0.2099 acc:0.9484
INFO:root:========= Testing: epoch[128/200] loss:0.2050 acc:0.9500
INFO:root:========= Testing: epoch[129/200] loss:0.1984 acc:0.9543
INFO:root:========= Testing: epoch[130/200] loss:0.2145 acc:0.9475
INFO:root:========= Testing: epoch[131/200] loss:0.2012 acc:0.9504
INFO:root:========= Testing: epoch[132/200] loss:0.2076 acc:0.9480
INFO:root:========= Testing: epoch[133/200] loss:0.2076 acc:0.9502
INFO:root:========= Testing: epoch[134/200] loss:0.2076 acc:0.9516
INFO:root:========= Testing: epoch[135/200] loss:0.2189 acc:0.9514
INFO:root:========= Testing: epoch[136/200] loss:0.2183 acc:0.9500
INFO:root:========= Testing: epoch[137/200] loss:0.2072 acc:0.9508
INFO:root:========= Testing: epoch[138/200] loss:0.2141 acc:0.